In [1]:
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.llms import Cohere
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQA
from langchain.vectorstores import Qdrant
import os
import pinecone
from tqdm.autonotebook import tqdm

c:\Users\Kwadwo Silas\Desktop\miscellenous\chatbot\env\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [8]:
os.environ["COHERE_API_KEY"] = "hv5YTaV6oUo5T9LGOY8F4bBtalGflhTU2FdPtEk3"
# initailize pinecode 

pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"),  # find at app.pinecone.io
    environment=os.getenv("PINECONE_ENV"),  # next to api key in console
)

In [2]:
# load our document here 
from langchain.document_loaders import TextLoader

loader = TextLoader(file_path="./gtbank_data_web.txt", encoding='utf8')

data = loader.load()

In [43]:
# load our document here 
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="./gtbank-data-web.csv")

data = loader.load()

In [44]:
# split texts into chunks

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1200,
    chunk_overlap  = 200,
    length_function = len,
)

data_chunks = text_splitter.split_documents(data)

In [46]:
len(data_chunks)

1913

In [9]:
# initialize embeddings 
embeddings = CohereEmbeddings(model = "multilingual-22-12")

In [47]:
# vector storage

db = Qdrant.from_documents(data_chunks, embeddings, location=":memory:", collection_name="my_documents", distance_func="Cosine")

In [74]:
# make our prompt 
prompt_template = """

generate response to the question based on the text provided.

Change instances where the bank is HDFC Bank to GTbank, 

If the text doesn't contain the answer, reply that the answer is not available and can request for more assistance by contacting us by telephone or sending a mail to customer service representative.

the Telephone Numbers:Tel: (+233) 302 611 560 Toll free: 0800 124 000 and the mail is  gh.customersupport@gtbank.com

Text: {context}

Question: {question}
"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

In [83]:
question = "how does the 737 platform works"

In [84]:
# question and answer here 
qa = RetrievalQA.from_chain_type(llm=Cohere(model="command-nightly", temperature=0), 
                                 chain_type="stuff", 
                                 retriever=db.as_retriever(),
                                                            
                                 chain_type_kwargs=chain_type_kwargs, 
                                 return_source_documents=True)
                                 

answer = qa({"query": question})

In [85]:
print(answer['result'])

 The 737 platform from GTbank works by allowing customers to open a bank account instantly by dialing *737*0# on their phone. 
Customers can also use the platform to carry out other banking activities like checking their account balance, transferring funds, and more. 
For more information or assistance, customers can contact GTbank customer service by telephone at Tel: (+233) 302 611 560 or toll-free at 0800 124 000, or by email at gh.customersupport@gtbank.com. 
